# Simple Pinecone setup with Llama-Index + Eval

In this example you will create a simple Llama Index RAG application and create the vector store in Pinecone. You'll also set up evaluation and logging with TruLens.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/quickstart/pinecone_quickstart.ipynb)

## Setup

### Install dependencies
Let's install some of the dependencies for this notebook if we don't have them already

In [ ]:
#! pip install trulens-eval==0.15.0 llama_index>=0.8.29post1 pinecone-client>=2.2.2 nltk>=3.8.1 html2text>=2020.1.16

### Add API keys
For this quickstart, you will need Open AI and Huggingface keys

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "..."
os.environ["PINECONE_API_KEY"] = "..."
os.environ["PINECONE_ENVIRONMENT"] = "..."

### Import from LlamaIndex and TruLens

In [ ]:
import pinecone
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores import PineconeVectorStore
from llama_index.llms import OpenAI
from llama_index import (
    VectorStoreIndex,
    SimpleWebPageReader,
    LLMPredictor,
    ServiceContext
)

from trulens_eval import TruLlama, Feedback, Tru, feedback
from trulens_eval.feedback import GroundTruthAgreement, Groundedness
tru = Tru()


### First we need to load documents. We can use SimpleWebPageReader

In [ ]:
# load documents
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["http://paulgraham.com/worked.html"]
)

Next we can create the vector store in pinecone.

In [ ]:
index_name = "paulgraham-essay"

# find API key in console at app.pinecone.io
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
# find ENV (cloud region) next to API key in console
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT')

# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT
)

In [ ]:
# create the index
pinecone.create_index(
        name=index_name,
        dimension=1536
    )

# set vector store as pinecone
vector_store = PineconeVectorStore(
        index_name=index_name,
        environment=os.environ["PINECONE_ENVIRONMENT"]
    )

In [ ]:
# set storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# set service context
llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm)

# create index from documents
index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
        service_context=service_context,
    )

### After creating the index, we can initilaize our query engine.

In [ ]:
query_engine = index.as_query_engine()

### Now we can set the engine up for evaluation and tracking

In [ ]:
import numpy as np

# Initialize OpenAI-based feedback function collection class:
openai = feedback.OpenAI()

# Define groundedness
grounded = Groundedness(groundedness_provider=openai)
f_groundedness = Feedback(grounded.groundedness_measure, name = "Groundedness").on(
    TruLlama.select_source_nodes().node.text # context
).on_output().aggregate(grounded.grounded_statements_aggregator)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance, name = "Answer Relevance").on_input_output()

# Question/statement relevance between question and each context chunk.
f_qs_relevance = Feedback(openai.qs_relevance, name = "Context Relevance").on_input().on(
    TruLlama.select_source_nodes().node.text
).aggregate(np.mean)

### Instrument query engine for logging with TruLens

In [ ]:
tru_query_engine_recorder = TruLlama(query_engine,
    app_id='LlamaIndex_App1',
    feedbacks=[f_groundedness, f_qa_relevance, f_qs_relevance])

In [ ]:
# Instrumented query engine can operate as a context manager:
with tru_query_engine_recorder as recording:
    llm_response = query_engine.query("What did the author do growing up?")
    print(llm_response)

### Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

### Or view results directly in your notebook

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all